# Нейросеть СNN  для распознавания тематики книги по обложке

In [1]:
import numpy as np
import keras
from keras.models import Model, Sequential
from keras.layers import LSTM, Conv2D, BatchNormalization, AveragePooling2D, Flatten, TimeDistributed
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.optimizers import RMSprop, Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import re
from random import shuffle
from glob import glob
import pandas as pd

all_files = glob('allfiles\\*.jpg')

In [3]:
all_f = []
for i in all_files:
    all_f.append(i.replace('allfiles'+'\\',''))
ff = pd.DataFrame(all_f, columns=['fname'])

ff - датафрейм чтобы проверить, что есть все файлы из загруженной таблицы; пустые записи удалим.

загрузим заранее подготовленный файл с классами и номером изображения

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import resample

subj_list = ['children', 'fantasy','medicine', 'music','mystery and detective stories', 
             'recipes', 'religion', 'romance', 'science','science fiction']

df = pd.read_csv('title_cover.csv')
df['path'] = df.file.apply(lambda x: 'allfiles'+'\\'+x )
df.head()

,children,fantasy,medicine,music,mystery and detective stories,recipes,religion,romance,science,science fiction,cover_id,file,path
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,575546.0,575546.jpg,allfiles\575546.jpg
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1981419.0,1981419.jpg,allfiles\1981419.jpg
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,905279.0,905279.jpg,allfiles\905279.jpg
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1321813.0,1321813.jpg,allfiles\1321813.jpg
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2964863.0,2964863.jpg,allfiles\2964863.jpg


In [5]:
books = df.merge(ff, left_on='file', right_on='fname', how='inner') 
# inner,  чтобы взять те файлы, которые точно есть и папке с файлами и в df

In [6]:
subj_new = ['children', 'fantasy','medicine', 'music','mystery and detective stories', 
             'religion', 'romance', 'science','science fiction']

In [7]:
books.drop_duplicates(inplace= True)

In [8]:
all_b = books[subj_new].sum()

In [41]:
w_lst = []
for i in range(0, len(subj_new)):
    w_lst.append(np.around(all_b[i]/all_b[0], decimals=1))

In [42]:
w_lst

[1.0, 1.7, 0.6, 1.4, 0.8, 3.4, 4.3, 4.1, 1.4]

In [11]:
X = books['path']
y = books[subj_new]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=888, shuffle = True)

In [13]:
Xtr, Xtst, ytr, ytst = list(np.array(X_train)), list(np.array(X_test)), np.array(y_train), np.array(y_test)

In [14]:
len(ytr)

28891

In [15]:
IMG_SIZE = (224, 224)  # размер входного изображения сети

In [17]:
def load_image(path, target_size=IMG_SIZE):
    img = load_img(path)  # загрузка и масштабирование изображения
    array = img_to_array(img)
    return preprocess_input(array)  # предобработка для ResNet50

# генератор для последовательного чтения обучающих данных с диска
def fit_generator(files, yy ,  batch_size = 32):    
    while True:
        #shuffle(files)
        for k in range(len(files) // batch_size):
            i = k * batch_size
            j = i + batch_size
            if j > len(files):
                j = - j % len(files)               
                
            x = np.array([load_image(path) for path in files[i:j]])
            y = yy[i:j]
            #print(i, j)
            #print(len(x))
            #print(y)           
            yield (x, y)

# генератор последовательного чтения тестовых данных с диска
def predict_generator(files):
    while True:
        for path in files:
            yield np.array([load_image(path)])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(20, 20))
for i in range(1, 11):
    subplot = fig.add_subplot(i // 5 + 1, 5, i)
    plt.imshow(plt.imread(Xtr[i]));
    book_title = []
    if ytr[i] !=0:
            #print(j, subj_list[j])
        book_title.append('religion')
    subplot.set_title('%s' %  str(book_title)); # поправить вывод названия
    
plt.show()

In [18]:
# base_model -  объект класса keras.models.Model (Functional Model)
base_model = ResNet50(include_top = True,
                   weights = 'imagenet',
                   input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3))

In [19]:
# фиксируем все веса предобученной сети
for layer in base_model.layers:
    layer.trainable = False

In [46]:
x = base_model.output 
x = keras.layers.Dropout(0.3) (x)
x = keras.layers.Dense(200, activation = 'relu') (x) # было 100
#x = keras.layers.Dropout(0.3) (x)
x = keras.layers.Dense(9,  # 9 выходов
                activation='sigmoid',  # функция активации  
                )(x) 

model = Model(inputs=base_model.input, outputs=x)

In [47]:
#model.summary();

In [48]:
model.compile(optimizer=Adam(), loss='binary_crossentropy',  metrics=['accuracy'])
checkp = ModelCheckpoint(filepath='weights.best.cover.hdf5', 
                               verbose=1, save_best_only=True)

In [36]:
train_val_split = 100  # число изображений в валидационной выборке
validation_data = next(fit_generator(Xtr[:train_val_split],ytr[:train_val_split], train_val_split))

In [37]:
#validation_data

In [38]:
# запускаем процесс обучения
model.fit_generator(fit_generator(Xtr[train_val_split:], ytr[train_val_split:], 100),  # данные читаем функцией-генератором
        steps_per_epoch=50,  # число вызовов генератора за эпоху
        epochs=20,# число эпох обучения
        class_weight =  np.array(w_lst)*100,
        validation_data=validation_data, callbacks = [checkp])

Epoch 1/20
49/50 [============================>.] - ETA: 0s - loss: 0.6171 - acc: 0.8610
Epoch 00001: val_loss improved from inf to 0.49098, saving model to weights.best.cover.hdf5
50/50 [==============================] - 23s 465ms/step - loss: 0.6146 - acc: 0.8615 - val_loss: 0.4910 - val_acc: 0.8856
Epoch 2/20
49/50 [============================>.] - ETA: 0s - loss: 0.3929 - acc: 0.8866
Epoch 00002: val_loss improved from 0.49098 to 0.34128, saving model to weights.best.cover.hdf5
50/50 [==============================] - 22s 444ms/step - loss: 0.3918 - acc: 0.8866 - val_loss: 0.3413 - val_acc: 0.8856
Epoch 3/20
49/50 [============================>.] - ETA: 0s - loss: 0.3299 - acc: 0.8869
Epoch 00003: val_loss improved from 0.34128 to 0.32699, saving model to weights.best.cover.hdf5
50/50 [==============================] - 22s 440ms/step - loss: 0.3299 - acc: 0.8869 - val_loss: 0.3270 - val_acc: 0.8856
Epoch 4/20
49/50 [============================>.] - ETA: 0s - loss: 0.3211 - acc: 0

In [39]:
model.load_weights('weights.best.cover.hdf5')

In [40]:
%%time
pred = model.predict_generator(predict_generator(Xtst), len(ytst), max_queue_size=100)

Wall time: 3min 2s


In [43]:
pred

array([[0.07815489, 0.05016572, 0.04485714, ..., 0.0547547 , 0.3870651 ,
        0.04640883],
       [0.06333856, 0.0729937 , 0.04096822, ..., 0.12319621, 0.31123066,
        0.0573553 ],
       [0.07813549, 0.02817815, 0.05179563, ..., 0.00591414, 0.7809319 ,
        0.02737962],
       ...,
       [0.0694932 , 0.12457986, 0.03562356, ..., 0.29124844, 0.08517682,
        0.13077694],
       [0.06647351, 0.08342028, 0.03914815, ..., 0.14669524, 0.19429107,
        0.08115155],
       [0.07825997, 0.02759339, 0.05210314, ..., 0.00540175, 0.79066473,
        0.02679958]], dtype=float32)

In [45]:
from sklearn.metrics import *
import sklearn as skl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
from pylab import rcParams

summary = classification_report(ytst, y_pred)
print(summary)

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       682
          1       0.00      0.00      0.00      1156
          2       0.00      0.00      0.00       411
          3       0.00      0.00      0.00       957
          4       0.00      0.00      0.00       510
          5       0.42      0.01      0.03      2266
          6       0.74      0.08      0.14      2887
          7       0.42      0.43      0.42      2774
          8       0.00      0.00      0.00       970

avg / total       0.34      0.11      0.13     12613



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(20, 20))
for i in range(1, 11):
    subplot = fig.add_subplot(i // 5 + 1, 5, i)
    plt.imshow(plt.imread(Xtr[i]));
    book_title = ()
    if ytr[i] !=0:
        book_title = 'religion. '+ str(np.around(pred[i],decimals= 2))
    else:
        book_title = str(np.around(pred[i], decimals = 2))
    subplot.set_title(book_title); # поправить вывод названия
    
plt.show()

In [ ]:
pred_class = np.around(pred)
pred_class

In [ ]:
from sklearn.metrics import *
import sklearn as skl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (10, 7)

summary = classification_report(ytst, pred_class)
print(summary)

In [ ]:
pred

In [ ]:
fpr, tpr, thresholds = roc_curve(ytst, pred)
fig = plt.figure()
plt.plot( fpr, tpr)    
plt.plot([0, 1], [0, 1])
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Book Subject classification. Religion')
plt.grid()
plt.show()
fig.savefig('Book Subject classification LSTM', dpi = 300, bbox_inches='tight')
print();